## Try 1


In [1]:
import os
import pathlib
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
import glob
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import splitfolders 

from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
image_path = pathlib.Path("../images")
DATA_DIR = pathlib.Path("../finetune_data")
train_path = DATA_DIR / "output/train"
val_path = DATA_DIR / "output/val"
test_path = DATA_DIR / "output/test"

In [3]:
Total_Images = glob.glob(f'{image_path}/*/*.jpg')
print("Total Number of Images", len(Total_Images))
Total_Images = pd.Series(Total_Images)

Total Number of Images 456


In [4]:
# sample visualization 

Total_Df = pd.DataFrame()

Total_Df['FileName'] = Total_Images.map(lambda ImageName :ImageName.split("H")[-1])

Total_Df['ClassId'] = Total_Images.map(lambda ImageName :ImageName.split("/")[-2])

Total_Df.head()

,FileName,ClassId
0,../images/extreme-close-up/casino.jpg,extreme-close-up
1,../images/extreme-close-up/psycho.jpg,extreme-close-up
2,../images/extreme-close-up/savingprivateryan.jpg,extreme-close-up
3,../images/extreme-close-up/capricorn-one.jpg,extreme-close-up
4,../images/extreme-close-up/goldfinger.jpg,extreme-close-up


In [5]:
# image counts 

Class_Id_Dist_Total = Total_Df['ClassId'].value_counts()
Class_Id_Dist_Total.head(10)

ClassId
close-up            145
medium-shot         140
long-shot           110
medium-long-shot     44
extreme-close-up     17
Name: count, dtype: int64

In [6]:
splitfolders.ratio(image_path, output=f"{DATA_DIR}/output", seed=101, ratio=(.8, .1, .1))

Copying files: 456 files [00:00, 2050.76 files/s]


In [7]:
train_path=f'{DATA_DIR}/output/train/'
val_path=f'{DATA_DIR}/output/val'
test_path=f'{DATA_DIR}/output/test'
class_names=os.listdir(train_path)
class_names_val=os.listdir(val_path)
class_names_test=os.listdir(test_path)

In [8]:
train_image1 = glob.glob(f'{DATA_DIR}/output/train/*/*.jpg')

Total_TrainImages = train_image1 
print("Total number of training images: ", len(Total_TrainImages))


test_image1 = glob.glob(f'{DATA_DIR}/output/test/*/*.jpg')

Total_TestImages = test_image1
print("Total number of test images: ", len(Total_TestImages))



Val_image1 = glob.glob(f'{DATA_DIR}/output/val/*/*.jpg')

Total_ValImages = Val_image1 
print("Total number of val images: ", len(Total_ValImages))

Total number of training images:  364
Total number of test images:  48
Total number of val images:  44


In [9]:
class_names = os.listdir(train_path)
print("Classes:", class_names)

# Load datasets
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

Classes: ['extreme-close-up', 'medium-shot', 'medium-long-shot', 'long-shot', 'close-up']


In [10]:
train_ds = image_dataset_from_directory(
    train_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

val_ds = image_dataset_from_directory(
    val_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

test_ds = image_dataset_from_directory(
    test_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

Found 364 files belonging to 5 classes.


2025-09-01 08:34:36.680628: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-09-01 08:34:36.680803: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-09-01 08:34:36.680807: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-09-01 08:34:36.681000: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-01 08:34:36.681012: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 44 files belonging to 5 classes.
Found 48 files belonging to 5 classes.


In [11]:
## OPTIONAL: preprocessing: crop images to better represent aspect ratio

def crop_to_aspect(image, aspect_ratio=1.5):
    shape = tf.shape(image)
    h, w = shape[0], shape[1]
    target_w = tf.cast(h * aspect_ratio, tf.int32)
    target_h = tf.cast(w / aspect_ratio, tf.int32)

    # If too wide, crop width
    if w > target_w:
        offset_w = (w - target_w) // 2
        image = tf.image.crop_to_bounding_box(image, 0, offset_w, h, target_w)
    else:  # Else, crop height
        offset_h = (h - target_h) // 2
        image = tf.image.crop_to_bounding_box(image, offset_h, 0, target_h, w)
    return image

def preprocess(image, label):
    image = crop_to_aspect(image, aspect_ratio=1.5)
    image = tf.image.resize(image, [224, 224])  # Or 299, etc.
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

TypeError: in user code:

    File "/var/folders/4r/pcfnz3yn2q537s45zk2swk7c0000gn/T/ipykernel_48605/604560806.py", line 19, in preprocess  *
        image = crop_to_aspect(image, aspect_ratio=1.5)
    File "/var/folders/4r/pcfnz3yn2q537s45zk2swk7c0000gn/T/ipykernel_48605/604560806.py", line 6, in crop_to_aspect  *
        target_w = tf.cast(h * aspect_ratio, tf.int32)

    TypeError: Expected int32 passed to parameter 'y' of op 'Mul', got 1.5 of type 'float' instead. Error: Expected int32, but got 1.5 of type 'float'.


In [12]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# Build model with ResNet50 base
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


2025-09-01 08:35:04.065425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 368ms/step - accuracy: 0.3348 - loss: 4.8637 - val_accuracy: 0.5909 - val_loss: 2.7634
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.4290 - loss: 3.9152 - val_accuracy: 0.5682 - val_loss: 2.3820
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 166ms/step - accuracy: 0.5196 - loss: 3.9221 - val_accuracy: 0.5682 - val_loss: 3.0352
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.6675 - loss: 3.0840 - val_accuracy: 0.5909 - val_loss: 2.3289
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.6375 - loss: 3.0801 - val_accuracy: 0.6364 - val_loss: 2.8673
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.6771 - loss: 2.0432 - val_accuracy: 0.5909 - val_loss: 2.4065
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - accuracy: 0.6495 - loss: 2.5856 - val_accuracy: 0.6364 - val_loss: 3.0114
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.6735 - loss: 2.3777 - val_accuracy: 0.6136 - val_

In [13]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 373ms/step - accuracy: 0.5139 - loss: 2.6485
Test accuracy: 0.52
